# Torch Adam - Constrained Optimization

In [ ]:
import optiland.backend as be
from optiland import optic, optimization

be.set_backend("torch")  # Set the backend to PyTorch
be.grad_mode.enable()  # Enable gradient tracking

Define a starting lens:

In [ ]:
lens = optic.Optic()

# add surfaces
lens.add_surface(index=0, thickness=be.inf)
lens.add_surface(index=1, thickness=7, radius=15, material="N-BK7", is_stop=True)
lens.add_surface(index=2, thickness=30, radius=-1000)
lens.add_surface(index=3)

# set aperture
lens.set_aperture(aperture_type="EPD", value=15)

# add field
lens.set_field_type(field_type="angle")
lens.add_field(y=0)
lens.add_field(y=5)
lens.add_field(y=10)

# add wavelength
lens.add_wavelength(value=0.45, is_primary=True)

# draw lens
_ = lens.draw(num_rays=5)

Define optimization problem:

In [ ]:
problem = optimization.OptimizationProblem()

Add operands (targets for optimization):

In [ ]:
# 1. add focal length operand
input_data = {"optic": lens}
problem.add_operand(operand_type="f2", target=50, weight=1, input_data=input_data)

# 2. add RMS spot size operand
input_data = {
    "optic": lens,
    "surface_number": -1,
    "Hx": 0,
    "Hy": 0,
    "num_rays": 5,
    "wavelength": 0.55,
    "distribution": "hexapolar",
}

problem.add_operand(
    operand_type="rms_spot_size",
    target=0,
    weight=1,
    input_data=input_data,
)

Define variables - constrain radius 1.

In [ ]:
problem.add_variable(lens, "radius", surface_number=1, min_val=10, max_val=30)

Let thickness to image plane vary.

In [ ]:
problem.add_variable(lens, "thickness", surface_number=2)

Check initial merit function value and system properties:

In [ ]:
problem.info()

Define optimizer:

In [ ]:
optimizer = optimization.TorchAdamOptimizer(problem)

Run optimization:

In [ ]:
res = optimizer.optimize(n_steps=100, lr=1e-8, gamma=0.99, disp=True)

Print merit function value and system properties after optimization:

In [ ]:
problem.info()

Draw final lens:

In [ ]:
_ = lens.draw(num_rays=5)